In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
nz = 10000
cell_size = 10.0
sat = 0.25
bot = 10.0
h = bot + sat * cell_size
top = bot + cell_size
deltaz = top - bot

In [ ]:
z = np.linspace(bot, h, nz + 1)

In [ ]:
ss = 1.0

In [ ]:
ss_fd = 0.0
for idx in range(nz):
    z0, z1 = z[idx], z[idx + 1]
    dz = z1 - z0
    rho = ss * dz
    psi = h - z0 - 0.5 * dz
    ss_fd += rho * psi
print(ss_fd)

In [ ]:
psi_avg = h - bot - deltaz * sat / 2
psi_avg

In [ ]:
ss * deltaz * sat * (h - bot - sat * (top - bot) / 2)

In [ ]:
ss * deltaz * sat * (h - bot)

In [ ]:
def quad_sat(x, tp, bt, omega=0.1):
    if isinstance(x, float):
        x = np.array([x], dtype=float)
    elif isinstance(x, (list, tuple)):
        x = np.array(x, dtype=float)

    y = np.zeros(x.shape, dtype=z.dtype)
    dz = tp - bt
    br = (x - bt) / dz
    br[x < bt] = 0.0
    br[x > tp] = 1.0

    if omega == 0:
        y[:] = br
    else:
        av = 1.0 / (1.0 - omega)

        idx = br < omega
        y[idx] = 0.5 * av * br[idx] ** 2.0 / omega

        idx = (br >= omega) & (br < 1.0 - omega)
        y[idx] = av * br[idx] + 0.5 * (1.0 - av)

        idx = br >= 1.0 - omega
        y[idx] = 1.0 - ((0.5 * av * (1.0 - br[idx]) ** 2) / omega)

    return y

In [ ]:
def quad_sat_derv_fd(x, tp, bt, omega=0.1, power=1):
    dx = 1e-6
    derv = np.zeros(x.shape, dtype=x.dtype)
    for idx, xx in enumerate(x):
        xx0, xx1 = xx - dx, xx + dx
        y0 = quad_sat(xx0, tp, bt, omega=omega)
        y1 = quad_sat(xx1, tp, bt, omega=omega)
        derv[idx] = (y1**power - y0**power) / (2 * dx)
    return derv

In [ ]:
harr = np.linspace(bot - 1.0, top + 1.0, nz)
omega = 1e-6

In [ ]:
sat_lin = quad_sat(harr, top, bot, omega=0.0)
sat_lin.mean()

In [ ]:
sat = quad_sat(harr, top, bot, omega=omega)
sat

In [ ]:
plt.plot(harr, sat_lin)
plt.plot(harr, sat)

In [ ]:
sat_derv = quad_sat_derv_fd(harr, top, bot, power=1, omega=omega)

In [ ]:
plt.plot(harr, sat_derv)

In [ ]:
plt.plot(harr, quad_sat_derv_fd(harr, top, bot, power=2, omega=omega))

In [ ]:
plt.plot(harr, 2 * sat * sat_derv)